# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [39]:
import pandas as pd
import numpy as np

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [40]:
df = pd.read_csv('fires-all.csv')
print(df.describe())

                 id    superficie           lat           lng  \
count  8.264000e+04  82640.000000  82616.000000  82616.000000   
mean   2.007259e+09     19.888085     41.763721     -5.664360   
std    4.247106e+06    223.787536      2.019672      4.394031   
min    2.001010e+09      1.000000      0.490720  -1000.000000   
25%    2.003390e+09      1.500000     40.876544     -7.257298   
50%    2.006330e+09      3.000000     42.371123     -6.019478   
75%    2.011150e+09      7.150000     43.143145     -4.303078   
max    2.015510e+09  28879.100000     87.824157    242.755603   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82640.000000  82640.000000  82640.000000  82640.000000   
mean          0.741626      7.850278     28.411375     77.795946   
std           0.437743      5.419922     11.727155     98.867769   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

## Preprocessing

### Tractament de missing values

In [41]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

['lat', 'lng', 'causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar lat i long

In [46]:
import corregirCoordenades
# idmunicipio 999, i 998 ens els carreguem ja que són incendis originats fora d'Espanya
df.drop(df.loc[df["idmunicipio"] == 999].index, inplace=True)
df.drop(df.loc[df["idmunicipio"] == 998].index, inplace=True)
print(df.columns[df.isnull().any()].tolist())

df = corregirCoordenades.correctCoords(df)

[]


In [ ]:
# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)

# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que
# Causa supuesta la posem a 0
df['causa_supuesta'].fillna(0, inplace=True)

In [ ]:
# Fem knn
df['gastos'].fillna(0, inplace=True)
df['perdidas'].fillna(0, inplace=True)

# df.describe()

print(df.columns[df.isnull().any()].tolist())

In [45]:
print(df.describe())

                 id    superficie           lat           lng  \
count  8.252300e+04  82523.000000  82523.000000  82523.000000   
mean   2.007258e+09     19.092067     41.719125     -5.557423   
std    4.246907e+06    207.085686      1.926842      2.443268   
min    2.001010e+09      1.000000     27.707180    -17.980518   
25%    2.003390e+09      1.500000     40.775439     -7.248247   
50%    2.006330e+09      3.000000     42.342616     -5.959559   
75%    2.011150e+09      7.100000     43.127795     -4.217251   
max    2.015510e+09  28879.100000     43.761264      4.158954   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82523.000000  82523.000000  82523.000000  82523.000000   
mean          0.640827      7.849969     28.412018     76.526471   
std           0.479761      5.419613     11.724082     92.866264   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

In [43]:
aux = df.loc[df['idmunicipio'] == 999]
latMissing = df.loc[np.isnan(df['lat'])]
print(aux.shape)
print(latMissing.shape)
print(latMissing)

(0, 21)
(0, 21)
Empty DataFrame
Columns: [id, superficie, fecha, lat, lng, latlng_explicit, idcomunidad, idprovincia, idmunicipio, municipio, causa, causa_supuesta, causa_desc, muertos, heridos, time_ctrl, time_ext, personal, medios, gastos, perdidas]
Index: []

[0 rows x 21 columns]


## Separem target del dataset
Volem predir la superfície cremada donat un incendi per tant separem aquesta variable de la resta.

In [44]:
X = df.drop('superficie', axis=1)
y = df['superficie']

# Comprovem que al target no hi hagin nulls
print(y.isnull().values.any())


False
